In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
import re

In [309]:
os.chdir('/Users/user/Desktop/ВКР/')

names = glob.glob('Archive/*')
names

['Archive/Овз•в ®о≠м 2011.xls',
 'Archive/Овз•в ђ†ав 2011.xls',
 'Archive/Овз•в п≠Ґ†ам 2012.xls',
 'Archive/Овз•в Ѓ™вп°ам 2011.xls',
 'Archive/Овз•в д•Ґа†Ђм 2011.xls',
 'Archive/Овз•в †Ґ£гбв 2011.xls',
 'Archive/Овз•в п≠Ґ†ам 2011.xls',
 'Archive/Овз•в ®оЂм 2011.xls',
 'Archive/Овз•в †ѓа•Ђм 2011.xls',
 'Archive/Овз•в ђ†© 2011.xls',
 'Archive/Овз•в б•≠вп°ам 2011.xls',
 'Archive/Овз•в §•™†°ам 2011.xls',
 'Archive/Овз•в ≠Ѓп°ам 2011.xls']

In [310]:
data = pd.DataFrame()
for name in names:
    df = pd.read_excel(name, names = ['time', 'screen', '?', 'sales', 'occupancy'], index_col = None)
    df = df.drop('?', axis = 1)
    df = df[~df['time'].isna()]
    data = pd.concat([data, df]).reset_index(drop=True)
    print('Ok')

Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok
Ok


In [311]:
films_idx = list(data[data['time'].str.startswith('Фильм:')].index) + [len(data)]
films_col = [None for i in range (films_idx[0])]

In [312]:
for i in range(1, len(films_idx)):
    film = data.iloc[films_idx[i-1]]['time']
    films_col = films_col + [film for _ in range(films_idx[i] - films_idx[i-1])]

In [313]:
data['film'] = films_col
data = data.dropna()
data = data[~data['sales'].str.contains('зрителей').fillna(False)]
data = data.reset_index(drop=True)

In [314]:
data['time'] = pd.to_datetime(data['time'])
data['sales'] = data['sales'].astype('int')
data['occupancy']  = data['occupancy'].astype('float')
data['screen'] = data['screen'].apply(lambda s: int(re.findall(r'-?\d+\.?\d*', s)[0]))

In [315]:
data['film'] = data['film'].apply(lambda x: x.replace('Фильм:', '').strip())

In [316]:
def clear_film(x):
    delete_list = ['3D', '(Digital)', 'digital', '3-ДЭ', 'DIGITAL', 'Digital', 'blu ray', 'в 4D', 'в 3 D']
    f = x
    for item in delete_list:
        f = f.replace(item, '').strip()
    return f

In [317]:
data['film'] = data['film'].apply(lambda x: clear_film(x))

In [318]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26357 entries, 0 to 26356
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       26357 non-null  datetime64[ns]
 1   screen     26357 non-null  int64         
 2   sales      26357 non-null  int64         
 3   occupancy  26357 non-null  float64       
 4   film       26357 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.0+ MB


In [319]:
data.to_csv('processed_schedule_data_2011.csv', index = False)

In [23]:
films_names = data['film'].unique()
films_names = list(set(films_names))
len(films_names)

238

# Parsing

In [149]:
import requests
from bs4 import BeautifulSoup

In [150]:
films_id_ = {}

In [151]:
fail = []

for film in films_names:
    try:
        url: str = 'https://www.kinopoisk.ru/index.php?kp_query={}/'.format(film)
        res: list[str] = requests.get(url)
        soup_kinopoisk: BeautifulSoup = BeautifulSoup(res.text, 'html')

        resps = soup_kinopoisk.find(class_='search_results')
        resps_children = list(resps.find(class_='name').children)
        movie_id = resps_children[0]['data-id']
        films_id_[film] = movie_id
    except: 
        fail.append(film)

In [152]:
len(fail)

41

In [153]:
fail2 = []
for film in fail:
    film_r = ' '.join(film.split(' ')[:-1])
    try:
        url: str = 'https://www.kinopoisk.ru/index.php?kp_query={}/'.format(film_r)
        res: list[str] = requests.get(url)
        soup_kinopoisk: BeautifulSoup = BeautifulSoup(res.text, 'html')
            
        resps = soup_kinopoisk.find(class_='search_results')
        resps_children = list(resps.find(class_='name').children)
        movie_id = resps_children[0]['data-id']
        films_id_[film] = movie_id
        
    except: 
        fail2.append(film)

In [154]:
len(fail2)

13

In [155]:
fail2

['Пираты Карибского моря: На странных берегах',
 'Три богатыря и шамаханская царица',
 'Я-четвертый',
 'Орёл девятого легиона',
 'Трансформеры 3: Темная сторона луны',
 'Ронал-Варвар',
 'Высоцкий. Спасибо, что живой',
 'Миссия невыполнима:Протокол Фантом',
 'Суперменеджер или Мотыга судьбы',
 'Криминальная фишка от Генри',
 'Ларго Винч 2: Заговор в Бирме',
 'Секс и Дзен : экстремальный экстаз',
 'Ржевский против Наполеона']

In [156]:
films_id_['Трансформеры 3: Темная сторона луны'] = '452899'
films_id_['Пираты Карибского моря: На странных берегах'] = '427076'
films_id_['Криминальная фишка от Генри'] = '463782'
films_id_['Высоцкий. Спасибо, что живой'] = '522036'
films_id_['Ронал-Варвар'] = '549198'
films_id_['4.3.2.1'] = '474776'
films_id_['Суперменеджер или Мотыга судьбы'] = '402980'
films_id_['Я-четвертый'] = '468872'
films_id_['Ржевский против Наполеона'] = '417854'
films_id_['Миссия невыполнима:Протокол Фантом'] = '472362'
films_id_['Секс и Дзен : экстремальный экстаз'] = '534911'
films_id_['Ларго Винч 2: Заговор в Бирме'] = '464606'
films_id_['Три богатыря и шамаханская царица'] = '432650'
films_id_['Орёл девятого легиона'] = '455353'
films_id_['Неуловимый Люк'] = '415322'
films_id_['Смешарики. Начало'] = '570117'
films_id_['Ключ саламандры'] = '453312'

In [157]:
len(films_id_)

238

In [253]:
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest
from kinopoisk_unofficial.request.staff.staff_request import StaffRequest
from kinopoisk_unofficial.request.films.box_office_request import BoxOfficeRequest

api_client = KinopoiskApiClient('783c58a4-397e-4cad-823b-b78c4a3cefc3')

In [159]:
IMDB_df = pd.read_csv('processed_IMDB_dataset.csv')

In [160]:
names_in_IMDB = list(set(IMDB_df['title']))

In [167]:
metadata_dict = {k:{'name': k, 
                    'belongs_to_collection':False,
                    'english_name': None,
                    'film_id': films_id_[k], 
                    'genres': None, 
                    'budget':None,
                    'currency': None,
                    'countries': None, 
                    'companies': None,
                    'film_length': None, 
                    'rating_await': None, 
                    'film_age': 0,
                    'actors' : None, 
                    'directors' : None, 
                    } for k in films_id_.keys()}

In [168]:
def get_actors_and_directors(response):
    actors = []
    directors = []
    for person in response.items[:5]:
        who = person.profession_text
        if who == 'Режиссеры':
            directors.append([person.name_ru, person.staff_id])
        elif who == 'Актеры':
            actors.append([person.name_ru, person.staff_id])
            
    return actors, directors

In [169]:
def get_budget(response):
    budget = 0
    currency = None
    for i in response.items:
        if i.type == 'BUDGET':
            budget += i.amount
            currency = i.currency_code
        elif i.type == 'MARKETING':
            budget += i.amount
            currency = i.currency_code
    return budget, currency

In [164]:
# film = 'Ёлки'
# id_ = 4694773

In [175]:
def get_movie_info(film):
    
    id_ = metadata_dict[film]['film_id']
    
    request = FilmRequest(id_)
    response = api_client.films.send_film_request(request)
    en_name = response.film.name_en or response.film.name_original

    if en_name in names_in_IMDB:
        metadata_dict[film]['english_name'] = en_name
        temp_dict = dict(IMDB_df[IMDB_df['title'] == en_name].iloc[0])
        metadata_dict[film]['belongs_to_collection'] = temp_dict['belongs_to_collection']
        metadata_dict[film]['genres'] = eval(temp_dict['genres'])
        metadata_dict[film]['budget'] = temp_dict['budget']
        metadata_dict[film]['countries'] = eval(temp_dict['production_countries'])
        metadata_dict[film]['companies'] = eval(temp_dict['production_companies'])
        metadata_dict[film]['film_length'] = temp_dict['runtime']
        metadata_dict[film]['rating_await'] = temp_dict['vote_count']
        metadata_dict[film]['film_age'] = 0 if not temp_dict['adult'] else 18
        metadata_dict[film]['actors'] = temp_dict['cast_ranking']
        metadata_dict[film]['directors'] = temp_dict['crew_ranking']

    else:
        metadata_dict[film]['genres'] = list(map(lambda x: x.genre, response.film.genres))
        metadata_dict[film]['countries'] = list(map(lambda x: x.country, response.film.countries))
        metadata_dict[film]['companies'] = []
        metadata_dict[film]['film_length'] = response.film.film_length
        metadata_dict[film]['rating_await'] = response.film.rating_await_count + response.film.rating_kinopoisk_vote_count
        metadata_dict[film]['film_age'] = response.film.rating_age_limits

        request2 = StaffRequest(id_)
        response2 = api_client.staff.send_staff_request(request2)
        metadata_dict[film]['actors'], metadata_dict[film]['directors'] = get_actors_and_directors(response2)

        request3 = BoxOfficeRequest(id_)
        response3 = api_client.films.send_box_office_request(request3)
        metadata_dict[film]['budget'], metadata_dict[film]['currency'] = get_budget(response3)

In [172]:
done_films = []

In [176]:
for film in metadata_dict:
    if film not in done_films: 
        get_movie_info(film)
        done_films.append(film)
        print(film)

Добыча
Ранго
Беременный
Я тоже тебя люблю
Механик
О чём еще говорят мужчины
Схватка
Вкус ночи
Один День
Любовь и другие лекарства
Generatiоn П
Всё в порядке, мама!
Страшно красив
Два дня
Я не знаю, как она делает это
Тачки 2
Зелёный шершень
Вой Банши
Зеленый фонарь
Бедная богатая девочка
Жених напрокат
Супер 8
История дельфина
Двойник дьявола
Хранитель времени
Кожа, в которой я живу
Аполлон 18
Племя
Шелуха
Нокаут
Аноним
Выкрутасы
Липучка
Прошлой ночью в Нью-Йорке
Фантом
Тайна красной планеты
Контрабанда
Восстание планеты обезьян
Боевой конь
Время ведьм
Дом грёз
О чем еще говорят мужчины
Медведь Йоги
Коломбиана
Служебный роман.Наше время
Кукарача
Ужасный Генри
Гномео и Джульетта
Грибы
Тихая застава
Турист
Потомки
Бурлеск
Близнецы-убийцы
Джейн Эйр
Палата
Люди Икс: Первый класс
Гениальный папа
Санктум
Медвежонок Винни и его друзья
Нечто
Пункт назначения 5
Прямо в сердце
Человек, который изменил все
Несносные боссы
Бабло
Реальная сказка
Большие Мамочки:Сын как отец
Опасный метод
Элвин и бу

In [185]:
movies_info = pd.DataFrame(metadata_dict).T

In [186]:
movies_info = movies_info.drop(['name'], axis = 1)
movies_info['belongs_to_collection'] = movies_info['belongs_to_collection'].apply(lambda x: 0 if not x else 1)

In [190]:
{x for k in movies_info['genres'] for x in k}

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'Thriller',
 'War',
 'Western',
 'биография',
 'боевик',
 'вестерн',
 'военный',
 'детектив',
 'детский',
 'для взрослых',
 'драма',
 'история',
 'комедия',
 'короткометражка',
 'криминал',
 'мелодрама',
 'музыка',
 'мультфильм',
 'приключения',
 'семейный',
 'спорт',
 'триллер',
 'ужасы',
 'фантастика',
 'фэнтези'}

In [191]:
translate_genres_dict = {'биография': 'History', 
                         'боевик': 'Action',
                         'вестерн': 'Western',
                         'военный': 'War',
                         'детектив': 'Mystery',
                         'детский': 'Animation',
                         'для взрослых': None,
                         'драма': 'Drama',
                         'история': 'History',
                         'комедия': 'Comedy',
                         'короткометражка': None,
                         'криминал': 'Crime',
                         'мелодрама': 'Romance',
                         'музыка': 'Music',
                         'мультфильм': 'Animation',
                         'приключения': 'Adventure',
                         'семейный': 'Family',
                         'спорт': None,
                         'триллер': 'Thriller',
                         'ужасы': 'Horror',
                         'фантастика': 'Science Fiction',
                         'фэнтези': 'Fantasy'}

In [200]:
movies_info['genres'] = movies_info['genres'].apply(lambda x: [translate_genres_dict.get(i, i) for i in x])
movies_info['genres'] = movies_info['genres'].apply(lambda x: [i for i in x if i])

In [212]:
movies_info['currency'] = movies_info['currency'].apply(lambda x: None if x == '' else x)
movies_info['currency'] = movies_info['currency'].fillna('USD')

In [213]:
movies_info['currency'].unique()

array(['USD', 'GBP', 'EUR', 'RUB', 'DKK'], dtype=object)

In [214]:
currency_courses_dict = {'USD': 1, 'GBP': 1.61, 'EUR': 1.38, 'RUB':0.04, 'DKK': 0.19}

In [215]:
movies_info['currency_course_to_dollars'] = movies_info['currency']. apply(lambda x: currency_courses_dict[x])

In [216]:
movies_info['budget_in_dollars'] = movies_info['budget'] * movies_info['currency_course_to_dollars']

In [220]:
{i for lst in movies_info['countries'] for i in lst}

{'Australia',
 'Austria',
 'Belgium',
 'Canada',
 'Czech Republic',
 'Denmark',
 'France',
 'Germany',
 'Hungary',
 'India',
 'Ireland',
 'Italy',
 'Mexico',
 'Netherlands',
 'New Zealand',
 'Poland',
 'Serbia and Montenegro',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Arab Emirates',
 'United Kingdom',
 'United States of America',
 'Австралия',
 'Аргентина',
 'Бельгия',
 'Великобритания',
 'Германия',
 'Гонконг',
 'Дания',
 'Индия',
 'Испания',
 'Италия',
 'Канада',
 'Китай',
 'Корея Южная',
 'Малайзия',
 'Нидерланды',
 'Россия',
 'СССР',
 'США',
 'Турция',
 'Украина',
 'Франция',
 'Швейцария',
 'Япония'}

In [221]:
translate_countries_dict = {'Австралия': 'Australia',
                             'Аргентина': 'Argentina',
                             'Бельгия': 'Belgium',
                             'Великобритания': 'United Kingdom',
                             'Германия': 'Germany',
                             'Гонконг': 'Hong Kong',
                             'Дания': 'Denmark',
                             'Индия': 'India',
                             'Испания': 'Spain',
                             'Италия': 'Italy',
                             'Канада': 'Canada',
                             'Китай': 'China',
                             'Корея Южная': 'South Korea',
                             'Малайзия': 'Malaysia',
                             'Нидерланды': 'Netherlands',
                             'Россия': 'Russia',
                             'СССР': 'Russia',
                             'США': 'United States of America',
                             'Турция': 'Türkiye',
                             'Украина': 'Ukraine',
                             'Франция': 'France',
                             'Швейцария': 'Switzerland',
                             'Япония': 'Japan'}

In [222]:
movies_info['countries'] = movies_info['countries'].apply(lambda x: [translate_countries_dict.get(i, i) for i in x])
movies_info['countries'] = movies_info['countries'].apply(lambda x: [i for i in x if i])

In [227]:
movies_info['actors_ranking'] = movies_info['actors'].apply(lambda x: x if isinstance(x, float) else None)

In [259]:
actors_id = list({k for d in movies_info[movies_info['actors_ranking'].isna()]['actors'].apply(lambda l: [x[-1] for x in l]) for k in d})
len(actors_id)

326

In [260]:
from kinopoisk_unofficial.request.staff.person_request import PersonRequest

In [261]:
def find_average_rating(id_):
        request = PersonRequest(id_)
        response = api_client.staff.send_person_request(request)
        lst_rank = list(map(lambda x: x.rating, response.films))
        m = np.array([float(x) for x in lst_rank if isinstance(x, str) ]).mean()
        return m

In [262]:
avg_rating = {}
i = 0

In [263]:
fail_id = []

In [264]:
while i < len(actors_id):
    id_ = actors_id[i]
    try: avg_rating[id_] = find_average_rating(id_)
    except: fail_id.append(id_)
    i+=1

In [265]:
len(fail_id)

186

In [281]:
for id_ in fail_id:
    avg_rating[id_] = 3

In [284]:
def get_raiting(d):
    l = [x[-1] for x in d]
    return np.mean(np.array([avg_rating[a] for a in l]))

In [291]:
movies_info['actors_ranking'] = movies_info['actors'].apply(lambda x: x if isinstance(x, float) else get_raiting(x))

In [292]:
movies_info['crew_ranking'] = movies_info['directors'].apply(lambda x: x if isinstance(x, float) else None)

In [293]:
dirs_id = list({k for d in movies_info[movies_info['crew_ranking'].isna()]['directors'].apply(lambda l: [x[-1] for x in l]) for k in d})
len(dirs_id)

116

In [294]:
avg_rating = {}
i = 0

In [295]:
fail_id = []

In [296]:
while i < len(dirs_id):
    id_ = dirs_id[i]
    try: avg_rating[id_] = find_average_rating(id_)
    except: fail_id.append(id_)
    i+=1

In [297]:
len(fail_id)

51

In [298]:
for id_ in fail_id:
    avg_rating[id_] = 3

In [300]:
def get_raiting_dirs(d):
    l = [x[-1] for x in d]
    return np.mean(np.array([avg_rating[a] for a in l]))

In [301]:
movies_info['crew_ranking'] = movies_info['directors'].apply(lambda x: x if isinstance(x, float) else get_raiting(x))

In [303]:
movies_info['companies'] = movies_info['companies'].apply(lambda x: ['other'] if not x else x)

In [306]:
movies_info.reset_index(inplace = True)

In [308]:
movies_info.to_csv('processed_russian_dataset.csv')